## Loading molecule from QCA and calculating the MM energy with a SMIRNOFF-format forcefield

In [ ]:
from openforcefield.topology import Molecule, Topology
from openforcefield.typing.engines.smirnoff.forcefield import ForceField
from openforcefield.utils import get_data_file_path
from forcebalance.molecule import Molecule as fb_molecule
from simtk import openmm, unit
import numpy as np
import os
import shutil
import sys
import pandas as pd 
from IPython.display import HTML
import base64
import json
import matplotlib.pyplot as plt
from io import BytesIO

### Define utility function to get energy of an OpenMM system 

#### Modifying Yudong's script

In [ ]:
def get_MM_energy(system, positions):
    """
    Return the potential energy.

    Parameters
    ----------
    system : simtk.openmm.System
        The system to check
    positions : simtk.unit.Quantity of dimension (natoms,3) with units of length
        The positions to use
        
    Returns
    ---------
    energy : simtk.unit.Quantity
        sum of total energy
    """

    integrator = openmm.VerletIntegrator(1.0 * unit.femtoseconds)
    context = openmm.Context(system, integrator)
    context.setPositions(positions)
    state = context.getState(getEnergy=True)
    energy = state.getPotentialEnergy().in_units_of(unit.kilocalorie_per_mole)
    return energy

def show_oemol_struc(oemol, torsions=False, atom_indices=[]):
    from IPython.display import Image
    from openeye import oedepict, oechem
    width = 400
    height = 300
        
    # Highlight element of interest
    class NoAtom(oechem.OEUnaryAtomPred):
        def __call__(self, atom):
            return False
    class AtomInTorsion(oechem.OEUnaryAtomPred):
        def __call__(self, atom):
            return atom.GetIdx() in atom_indices
    class NoBond(oechem.OEUnaryBondPred):
        def __call__(self, bond):
            return False
    class BondInTorsion(oechem.OEUnaryBondPred):
        def __call__(self, bond):
            return (bond.GetBgn().GetIdx() in atom_indices) and (bond.GetEnd().GetIdx() in atom_indices)
    class CentralBondInTorsion(oechem.OEUnaryBondPred):
        def __call__(self, bond):
            return (bond.GetBgn().GetIdx() in atom_indices[1:3]) and (bond.GetEnd().GetIdx() in atom_indices[1:3])

        
    opts = oedepict.OE2DMolDisplayOptions(
                        width, height, oedepict.OEScale_AutoScale
                    )
    opts.SetAtomPropertyFunctor(oedepict.OEDisplayAtomIdx())
    
    oedepict.OEPrepareDepiction(oemol)
    img = oedepict.OEImage(width, height)
    display = oedepict.OE2DMolDisplay(oemol, opts)
    if(torsions):
        atoms = oemol.GetAtoms(AtomInTorsion())
        bonds = oemol.GetBonds(NoBond())
        abset = oechem.OEAtomBondSet(atoms, bonds)
        oedepict.OEAddHighlighting(display, oechem.OEColor(oechem.OEYellow), oedepict.OEHighlightStyle_BallAndStick, abset)
    
    oedepict.OERenderMolecule(img, display)
    png = oedepict.OEWriteImageToString("png", img)
    return Image(png)

def show_id(num):
    import os
    import json
    sdf_filename = '/home/maverick/Desktop/OpenFF/openforcefield-forcebalance/fit0/targets/QCP-'+str(num)+'/mol.sdf'
    mol = Molecule.from_file(sdf_filename, allow_undefined_stereo=True)
    oemol = mol.to_openeye()
    with open(os.path.dirname(sdf_filename)+'/metadata.json') as json_file:
        data = json.load(json_file)
        atom_indices = data["dihedrals"][0]
    return show_oemol_struc(oemol, torsions=True, atom_indices=atom_indices)

os.environ['OMP_NUM_THREADS'] = '1'

def build_context(offxml, molfile, param_id, dihedrals):
    """ Build an OpenMM Context from a offxml file and a molecule file """
    forcefield = ForceField(offxml, allow_cosmetic_attributes=True)
    forcefield = zero_out_torsion_param(forcefield, param_id)
    molecule = Molecule.from_file(molfile, allow_undefined_stereo=True)
    system = forcefield.create_openmm_system(molecule.to_topology())
    for i in dihedrals:
        system.setParticleMass(i, 0.0)
    integrator = openmm.LangevinIntegrator(
            300.0 * unit.kelvin,
            1.0 / unit.picosecond,
            2.0 * unit.femtosecond)
    platform = openmm.Platform.getPlatformByName('Reference')
    context = openmm.Context(system, integrator, platform)
    return context

def evaluate_minimized_energies(context, trajectory):
    """ Evaluate energies of all frames in a trajectory using OpenMM context """
    energies = []
    for frame_geo in trajectory:
        context.setPositions(frame_geo * unit.angstrom)
        openmm.LocalEnergyMinimizer_minimize(context, tolerance=5.0E-9, maxIterations=1500)
        # get minimized energy
        energy = context.getState(getEnergy=True).getPotentialEnergy()
        energy = energy.value_in_unit(unit.kilocalories_per_mole)
        energies.append(energy)
    return np.array(energies)

def plot_energies_data(energies_data_dict, title):
    """ Generate an energy data plot fromt data dict """
    plt.Figure(figsize=(3, 3))
    x_axis = energies_data_dict.pop('td_angles')
    for dataname, datavalues in energies_data_dict.items():
        if dataname=='QM':
            plt.plot(x_axis, datavalues, 'D-', label=dataname, markersize=7)
        else:
            dataname = 'QM - ' + dataname + '_intrinsic'
            plt.plot(x_axis, datavalues, 'o-', label=dataname)
    plt.legend()
    plt.title(title+"  Residuals")
    plt.ylabel("Residuals (QM - MM_intrinsic) [ kcal/mol ]")
    plt.xlabel("Torsion Angle [ degree ]")
    plot_iobytes = BytesIO()
    plt.savefig(plot_iobytes, format='png', dpi=100)
    plt.close()
    return(plot_iobytes)

def fig2inlinehtml(fig):
    figfile = fig.data
    # for python 3.x:
    figdata_png = base64.b64encode(figfile).decode()
    imgstr = '<img src="data:image/png;base64,{}" />'.format(figdata_png)
    return imgstr

def get_assigned_torsion_param(molecule, ff, dihedrals):
    
    topology = Topology.from_molecules([molecule])
    # Run the molecule labeling
    forcefield = ForceField(ff, allow_cosmetic_attributes=True)
    molecule_force_list = forcefield.label_molecules(topology)
    
    # Print out a formatted description of the parameters applied to this molecule
    for mol_idx, mol_forces in enumerate(molecule_force_list):
        for force_tag, force_dict in mol_forces.items():
            if force_tag == "ProperTorsions":
                for (atom_indices, parameter) in force_dict.items():
                    if atom_indices == tuple(dihedrals) or tuple(reversed(atom_indices)) == tuple(dihedrals):
                        return parameter.id

def zero_out_torsion_param(forcefield, param_id):
    if hasattr(forcefield.get_parameter_handler('ProperTorsions').get_parameter({'id':param_id})[0], '_k'):
        dim = len(forcefield.get_parameter_handler('ProperTorsions').get_parameter({'id':param_id})[0]._k)
        for i in range(dim):
            forcefield.get_parameter_handler('ProperTorsions').get_parameter({'id':param_id})[0]._k[i] = unit.Quantity(value=0, unit=unit.kilocalorie_per_mole)
        return forcefield
    elif hasattr(forcefield.get_parameter_handler('ProperTorsions').get_parameter({'id':param_id})[0], '_k_bondorder'):
        dim = len(forcefield.get_parameter_handler('ProperTorsions').get_parameter({'id':param_id})[0]._k_bondorder)
        for i in range(dim):
            forcefield.get_parameter_handler('ProperTorsions').get_parameter({'id':param_id})[0]._k_bondorder[i][1] = unit.Quantity(value=0, unit=unit.kilocalorie_per_mole)
            forcefield.get_parameter_handler('ProperTorsions').get_parameter({'id':param_id})[0]._k_bondorder[i][2] = unit.Quantity(value=0, unit=unit.kilocalorie_per_mole)
        return forcefield

## Load a molecule with positions, and evaluate its energy

In [ ]:
with open('TIG_targets.txt') as f:
    subdirs = f.readlines()

output_folder = 'compare_ff_fit7_tig5a'
if os.path.isdir(output_folder):
    shutil.rmtree(output_folder)
os.mkdir(output_folder)

offxml_list = ['/home/maverick/Desktop/OpenFF/openforcefield-forcebalance/fit7/result/optimize/fit7.offxml', \
               'openff_unconstrained-1.3.0.offxml']
ff_names = [os.path.splitext(os.path.basename(f))[0] for f in offxml_list]

df = pd.DataFrame(columns = ['Torsion ID', 'assigned params', 'Max - min for (QM - MM_fit7) kcal/mol', 'Max - min for (QM - MM_1.3.0) kcal/mol', 'Chemical Structure', 'QM-MM_intrinsic relative energies']) 

In [ ]:
count = 0
for mol_folder in subdirs:
    count += 1
#     if(count > 10):
#         break
    mol_folder = mol_folder.rstrip()
    if (count % 10 == 0):
        print("processed ", count)
    count += 1
    mol_file = mol_folder+'/mol.sdf'
    mol = Molecule.from_file(mol_folder+'/mol.sdf', allow_undefined_stereo=True)
    # build openmm contexts for each force field file

    # find scan trajectories
    traj_files = [f for f in os.listdir(mol_folder) if os.path.splitext(f)[-1] == '.xyz']
    # create output subfolder
    mol_name = os.path.basename(mol_folder)
    with open(mol_folder+'/metadata.json') as json_data:
            metadata = json.load(json_data)
    dihedrals = metadata["dihedrals"][0]
    assigned_torsion_params = dict((os.path.splitext(os.path.basename(key))[0], get_assigned_torsion_param(mol, key, dihedrals)) for key in offxml_list)
#     {key: value for (key, value) in iterable}[ for x in offxml_list]

#   zero out the assigned torsion parameter for this particular dihedral to get the intrinsic torsion potential 
#   "(full QM energy) - (full MM energy, locally optimized, with that specific torsion zeroed out)"
    contexts = []
    for i in range(len(offxml_list)):
        key = os.path.splitext(os.path.basename(offxml_list[i]))[0]
        contexts.append(build_context(offxml_list[i], mol_file, assigned_torsion_params[key], dihedrals))
        
    for f in traj_files:
#         print(f"- {f}")
        # hold energy data for this trajectory
        energies_data_dict = {}
        # use ForceBalance.molecule to read the xyz file
        fb_mol = fb_molecule(os.path.join(mol_folder, f))
        # read torsion angles
        with open(mol_folder+'/metadata.json') as json_data:
            metadata = json.load(json_data)
        energies_data_dict['td_angles'] = metadata["torsion_grid_ids"]
        # read QM energies
        qdata = np.genfromtxt(mol_folder+'/qdata.txt', delimiter="ENERGY ", dtype=None)
        #print(qdata.shape)
        eqm = qdata[:, 1]
        # record the index of the minimum energy structure
        ground_idx = np.argmin(eqm)
        eqm = [627.509*(x - eqm[ground_idx]) for x in eqm]
        energies_data_dict['QM'] = eqm
        
        # evalute mm energies for each force field
        for context, ffname in zip(contexts, ff_names):
            mm_energies = evaluate_minimized_energies(context, fb_mol.xyzs)
            # mm_energies -= mm_energies[ground_idx]
            mm_energies -= mm_energies.min()
            energies_data_dict[ffname] = mm_energies
            energies_data_dict[ffname] = energies_data_dict['QM'] - energies_data_dict[ffname]
        # plt the dataat the end of a row
#         for key, value in energies_data_dict.items():
#             print(key, value)
            
        plot_iobytes = plot_energies_data(energies_data_dict, mol_name)
        figdata_png = base64.b64encode(plot_iobytes.getvalue()).decode()
        plot_str = '<img src="data:image/png;base64,{}" />'.format(figdata_png)
        struc_str = fig2inlinehtml(show_id(mol_name[4:]))
        qm_fit7 = np.max(energies_data_dict['fit7']) - np.min(energies_data_dict['fit7'])
        qm_fit130 = np.max(energies_data_dict['openff_unconstrained-1.3.0']) - np.min(energies_data_dict['openff_unconstrained-1.3.0'])
        info_dict = {}
        tid = mol_name[4:]
        info_dict['assigned_params'] = assigned_torsion_params
        
        df = df.append({'Torsion ID': tid, 
                        'assigned params': info_dict,
                    'Max - min for (QM - MM_fit7) kcal/mol': qm_fit7, 
                    'Max - min for (QM - MM_1.3.0) kcal/mol': qm_fit130, 
                    'Chemical Structure': struc_str, 
                    'QM-MM_intrinsic relative energies': plot_str}, 
                    ignore_index=True)

## Here is a dataframe that is sorted by the averaged absolute difference between the conformer energies (QM - MM_fit0) in ascending order

In [ ]:
# # print dataframe. 
sorted_by_fit7_diff = df.sort_values('Max - min for (QM - MM_fit7) kcal/mol')
HTML(sorted_by_fit7_diff.to_html(escape=False))

In [ ]:
df.to_pickle("./residual_data.pkl", compression='gzip')

In [ ]:
df.to_html('residual_data.html')

In [ ]:
len(df)

In [ ]:
df = pd.read_pickle("./compare_ff_data.pkl", compression='gzip')

In [ ]:
fit0_list = df['Avg. abs(QM - MM_fit0) kcal/mol'].to_list()
fit3_list = df['Avg. abs(QM - MM_fit3) kcal/mol'].to_list()
fit_list = df['Avg. abs(QM - MM_1.3.0) kcal/mol'].to_list()
xval = list(zip(fit0_list, fit3_list, fit_list))
x_val = np.array(xval)


In [ ]:
plt.figure()
n_bins = 30
plt.hist(x_val, n_bins, density=True, histtype='step', stacked=False, fill=False, label=['fit0', 'fit3', '1.3.0'])
plt.xlabel("Average of |QM - MM| in kcal/mol ")
plt.ylabel("Density")
plt.legend()
plt.show()

In [ ]:
for tid in ['t43', 't44', 't45', 't47', 't69', 't69a', 't76', 't77', 't78']:
    count = 0
    fit0_diff = 0
    ff130_diff = 0
    for i in range(len(df)):
        dict_obj = df.iloc[i]['assigned params'] 
        if dict_obj['assigned_params']['openff_unconstrained-1.3.0'] == tid:
            count += 1
            fit0_diff += df.iloc[i]['Avg. abs(QM - MM_fit0) kcal/mol'] 
            ff130_diff += df.iloc[i]['Avg. abs(QM - MM_1.3.0) kcal/mol'] 
    if (count != 0):
        print(tid, count, "Fit0: ", '{0:.2f}'.format(fit0_diff/count), ", ff_1.3.0:", '{0:.2f}'.format(ff130_diff/count), ", drop from 1.3.0:", '{0:.2f}'.format((ff130_diff - fit0_diff)/count), "in kcal/mol")
    else:
        print("no matches for ", tid)

In [ ]:
df.iloc[1]['assigned params']['assigned_params']

In [ ]:
len(df)